In [ ]:
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt

########################SET folder to path of code directory in the project folder#####################
## SET PATH to code directory in the project folder
code_path = "C:\\Users\\evdoxiataka\\codes\\effi_user_study_1\\code\\"
import sys  
sys.path.insert(1, code_path)

########################SET folder to project directory path#####################
folder= "C:\\Users\\evdoxiataka\\codes\\effi_user_study_1\\"

from utils.data_loading import get_all_participants_logs, get_all_participants_feedback

import warnings
from pandas.errors import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

# Data Loading

In [ ]:
## LOAD FEEDBACK
prolific_export_filePath = folder+"data\\collected_data\\prolific_export_demographics.csv"
interaction_logs_filePath = folder+"data\\collected_data\\LOGS\\"
logs_df = get_all_participants_logs(prolific_export_filePath, interaction_logs_filePath)
feedback_df = get_all_participants_feedback(logs_df)
# logs_df.to_csv("Feedback_final.csv")

## LOAD APPLICATIONS SHOWN TO PARTICIPANTS
applications_df = pd.read_csv(folder+"data\\processed_data\\Applications_Converted.csv")
ft_weight_dict = {}

for i in range(0,49):
    ft_weight_dict.update({applications_df["ft_name_"+ str(i)][0]:applications_df["ft_weight_"+ str(i)][0]})

df = applications_df[["Application_id","Predicted_decision", "Prediction_Confidence_Accepted" ]]
df = df.T
df.rename(columns=df.iloc[0],inplace =True)

# Analysis of Participants' Interaction with the UI components

## Average number of clicks that a user did on a function with std

In [ ]:
function_group = logs_df.groupby(by = ["ID","Function"])

function_count = function_group.size().unstack(fill_value=0).stack()

parti_list = []
function_list = []
function_count_list = function_count.tolist()

# print(len(function_count_list))

for i in range(0,len(function_count.index)):
    parti_list.append(function_count.index[i][0])
    function_list.append(function_count.index[i][1])

parti_func_df = pd.DataFrame()

parti_func_df["Participant"] = parti_list
parti_func_df["Function"] = function_list
parti_func_df["Participant_Function_Count"] = function_count_list
parti_func_df

In [ ]:
data = []

for func in parti_func_df["Function"].unique():
    
    data.append(
    {
        "Function" : func,
        "Total" : parti_func_df[parti_func_df["Function"]== func ].Participant_Function_Count.sum(),
        "Mean": round(parti_func_df[parti_func_df["Function"]== func ].Participant_Function_Count.mean(),2),
        "Std": round(parti_func_df[parti_func_df["Function"]== func ].Participant_Function_Count.std(),2),
        "Number of Participants Clicked" : parti_func_df[(parti_func_df["Function"]== func) & (parti_func_df["Participant_Function_Count"] != 0)].shape[0]
    }
    )
    
each_function_df = pd.DataFrame(data)
each_function_df.sort_values(by='Total', ascending=False, inplace = True)
each_function_df.reset_index(drop = True, inplace = True)
each_function_df

each_function_df.replace({'Function' : { "select_applications" :'Select an application in APPs table', 
                                        "apply_refine_search":"Filter for an attribute in APPs table",
                                        "similar_application": "Selecting an Application in Scatter plot (in Compare)",
                                        "Similar_Application_table_Similarity_of_each_attribute": "Sorting the table on Similarity of each attribute  (in Compare)",
                                        "select_reject_application":'Give rating and/or weight for an Application',
                                        "Applications_Prediction_Confidence":"Sorting Applications Prediction Confidence",
                                        "Click on Feature Combination Button":"Click on Feature Combination Button",
                                       "Clicked-on-Causal-Graph-Node":"Click on node of causal graph"
                                       }},inplace=True)
each_function_df

In [ ]:
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])


bars = ax.bar(each_function_df["Function"],each_function_df["Total"], width=0.5, color = "green")
ax.bar_label(bars)
plt.title("Clicks vs Function")
plt.xlabel('Function')
plt.ylabel('Number of Clicks')
fig.autofmt_xdate()
plt.show()

## How many times each application was viewed w.r.t each user

In [ ]:
logs_df[logs_df["Function"] == "select_applications"]

display(logs_df[logs_df["Function"] == "select_applications"].head())
print(logs_df[logs_df["Function"] == "select_applications"]["ID"].nunique())

In [ ]:
app_group = logs_df[logs_df["Function"] == "select_applications"].groupby(by = ["ID","App ID"])

app_count = app_group.size().unstack(fill_value=0).stack()

data = []

for i in range(0,len(app_count.index)):    
    data.append(
    {
        "Participant" : app_count.index[i][0],
        "App ID" : app_count.index[i][1],
        "Count": app_count.values[i]
    }
    )
    
parti_app_df = pd.DataFrame(data)
parti_app_df

In [ ]:
data = []

for app_id in parti_app_df["App ID"].unique():    
    data.append(
    {
        "Application" : app_id,
        "Total" : parti_app_df[parti_app_df["App ID"]== app_id ].Count.sum(),
        "Mean": round(parti_app_df[parti_app_df["App ID"]== app_id ].Count.mean(),2),
        "Std": round(parti_app_df[parti_app_df["App ID"]== app_id ].Count.std(),2),
        "Number of Participants Viewed" : parti_app_df[(parti_app_df["App ID"]== app_id) & (parti_app_df["Count"] != 0)].shape[0]
    }
    )
    
each_app_df = pd.DataFrame(data)
each_app_df

each_app_df.sort_values(by='Total', ascending=False, inplace = True)
each_app_df.fillna("NA", inplace = True)
each_app_df.reset_index(drop = True, inplace = True)
each_app_df

# each_app_df.to_csv("How many times each application was viewed w.r.t each user.csv")

## Which attribute was most filtered for w.r.t each participant

In [ ]:
logs_df[logs_df["Function"] == "apply_refine_search"]

display(logs_df[logs_df["Function"] == "apply_refine_search"].head())
print("No.Part.:",logs_df[logs_df["Function"] == "apply_refine_search"]["ID"].nunique())

In [ ]:
attribute_group = logs_df[logs_df["Function"] == "apply_refine_search"].groupby(by = ["ID","Attribute"])

attribute_count = attribute_group.size().unstack(fill_value=0).stack()

data = []

for i in range(0,len(attribute_count.index)):
    
    if attribute_count.index[i][1] in list(ft_weight_dict.keys()):
        data.append(
        {
            "Participant" : attribute_count.index[i][0],
            "Attribute" : attribute_count.index[i][1],
            "Count": attribute_count.values[i]
        }
        )
    
parti_attri_df = pd.DataFrame(data)
parti_attri_df

In [ ]:
data = []

for attribute in list(ft_weight_dict.keys()):
    
    data.append(
    {
        "Attribute" : attribute,
        "Total" : parti_attri_df[parti_attri_df["Attribute"]== attribute ].Count.sum(),
        "Mean": round(parti_attri_df[parti_attri_df["Attribute"]== attribute ].Count.mean(),2),
        "Std": round(parti_attri_df[parti_attri_df["Attribute"]== attribute ].Count.std(),2),
        "Number of Participants Filtered" : parti_attri_df[(parti_attri_df["Attribute"]== attribute) & (parti_attri_df["Count"] != 0)].shape[0]
    }
    )
    
each_attri_df = pd.DataFrame(data)
each_attri_df

each_attri_df.sort_values(by='Total', ascending=False, inplace = True)
each_attri_df.fillna("NA", inplace = True)
each_attri_df.reset_index(drop = True, inplace = True)
each_attri_df
# each_attri_df.to_csv("Attribute was most filtered for w.r.t each participant.csv")

## Applications_Prediction_Confidence : Ascended or Descended w.r.t each User

In [ ]:
# logs_df[logs_df["Function"] == "Applications_Prediction_Confidence"]

In [ ]:
pc_group= logs_df[logs_df["Function"] == "Applications_Prediction_Confidence"].groupby(by = ["ID","Value"])

pc_count = pc_group.size().unstack(fill_value=0).stack()

data = []

for i in range(0,len(pc_count.index)):
    
    data.append(
    {
        "Participant" : pc_count.index[i][0],
        "PC_Value" : pc_count.index[i][1],
        "Count": pc_count.values[i]
    }
    )
    
parti_pc_df = pd.DataFrame(data)
parti_pc_df

In [ ]:
data = []

for pc_action in logs_df[logs_df["Function"] == "Applications_Prediction_Confidence"]["Value"].unique():
    
    data.append(
    {
        "PC_Value" : pc_action,
        "Total" : parti_pc_df[parti_pc_df["PC_Value"]== pc_action ].Count.sum(),
        "Mean": round(parti_pc_df[parti_pc_df["PC_Value"]== pc_action ].Count.mean(),2),
        "Std": round(parti_pc_df[parti_pc_df["PC_Value"]== pc_action ].Count.std(),2),
        "Number of Participants Clicked" : parti_pc_df[(parti_pc_df["PC_Value"]== pc_action) & (parti_pc_df["Count"] != 0)].shape[0]
    }
    )
    
each_pc_df = pd.DataFrame(data)

each_pc_df.sort_values(by='Total', ascending=False, inplace = True)
each_pc_df.fillna("NA", inplace = True)
each_pc_df.reset_index(drop = True, inplace = True)
display(each_pc_df)

## How many participants selected to view a similar application?

In [ ]:
p_ids_similar = logs_df[logs_df["Function"] == "similar_application"]['ID'].unique()

In [ ]:
len(p_ids_similar)

## How many participants clicked on Feature Combinations buttons

In [ ]:
logs_df[logs_df["Function"] == "Click on Feature Combination Button"]['ID'].nunique()


## Which attribute was most clicked on the causal graph w.r.t each participant

In [ ]:
logs_df[logs_df["Function"] == "Clicked-on-Causal-Graph-Node"]

display(logs_df[logs_df["Function"] == "Clicked-on-Causal-Graph-Node"].head())
print("No. Part.:",logs_df[logs_df["Function"] == "Clicked-on-Causal-Graph-Node"]["ID"].nunique())

In [ ]:
attribute_group = logs_df[logs_df["Function"] == "Clicked-on-Causal-Graph-Node"].groupby(by = ["ID","Value"])

attribute_count = attribute_group.size().unstack(fill_value=0).stack()

data = []

for i in range(0,len(attribute_count.index)):
    
    if attribute_count.index[i][1] in list(ft_weight_dict.keys()):
        data.append(
        {
            "Participant" : attribute_count.index[i][0],
            "Attribute" : attribute_count.index[i][1],
            "Count": attribute_count.values[i]
        }
        )
    
parti_attri_df = pd.DataFrame(data)
parti_attri_df

In [ ]:
data = []

for attribute in list(ft_weight_dict.keys()):
    
    data.append(
    {
        "Attribute" : attribute,
        "Total" : parti_attri_df[parti_attri_df["Attribute"]== attribute ].Count.sum(),
        "Mean": round(parti_attri_df[parti_attri_df["Attribute"]== attribute ].Count.mean(),2),
        "Std": round(parti_attri_df[parti_attri_df["Attribute"]== attribute ].Count.std(),2),
        "Number of Participants Clicked Node" : parti_attri_df[(parti_attri_df["Attribute"]== attribute) & (parti_attri_df["Count"] != 0)].shape[0]
    }
    )
    
each_attri_df = pd.DataFrame(data)
each_attri_df

each_attri_df.sort_values(by='Total', ascending=False, inplace = True)
each_attri_df.fillna("NA", inplace = True)
each_attri_df.reset_index(drop = True, inplace = True)
each_attri_df
# each_attri_df.to_csv("Attribute was most clicked on causal graph w.r.t each participant.csv")

# Analysis of Feedback

## How many applications were rated as fair/unfair w.r.t each participant

In [ ]:
rating_group= feedback_df[feedback_df["Function"] == "select_reject_application"].groupby(by = ["ID","Attribute"])

rating_count = rating_group.size().unstack(fill_value=0).stack()


data = []

for i in range(0,len(rating_count.index)):
    
    data.append(
    {
        "Participant" : rating_count.index[i][0],
        "Rating" : rating_count.index[i][1],
        "Count": rating_count.values[i]
    }
    )
    
parti_rating_df = pd.DataFrame(data)
parti_rating_df

In [ ]:
data = []

for rating in parti_rating_df["Rating"].unique():
    
    data.append(
    {
        "Rating" : rating,
        "Total" : parti_rating_df[parti_rating_df["Rating"]== rating ].Count.sum(),
        "Mean": round(parti_rating_df[parti_rating_df["Rating"]== rating ].Count.mean(),2),
        "Std": round(parti_rating_df[parti_rating_df["Rating"]== rating ].Count.std(),2),
        "Number of Participants Clicked" : parti_rating_df[(parti_rating_df["Rating"]== rating) & (parti_rating_df["Count"] != 0)].shape[0]

    }
    )
    
each_rating_df = pd.DataFrame(data)

each_rating_df.sort_values(by='Total', ascending=False, inplace = True)
each_rating_df.fillna("NA", inplace = True)
each_rating_df.reset_index(drop = True, inplace = True)
each_rating_df
# each_rating_df.to_csv("select_reject_application.csv")

## How often participants gave unfair labels w.r.t each participant

In [ ]:
parti_rating_df2 = parti_rating_df[parti_rating_df['Rating']=='unfair']
data = []

for participant in parti_rating_df2["Participant"].unique():
    
    data.append(
    {
        "Participant" : participant,
        "Total_unfair_Ratings" : parti_rating_df2[parti_rating_df2["Participant"]== participant].Count.sum(),
        # "Mean": round(parti_rating_df[parti_rating_df["Participant"]== participant].Count.mean(),2),
        # "Std": round(parti_rating_df[parti_rating_df["Participant"]== participant].Count.std(),2)
    }
    )    
    
each_participant_rating_df = pd.DataFrame(data)
each_participant_rating_df.sort_values(by='Total_unfair_Ratings', ascending=False, inplace = True)
each_participant_rating_df.fillna("NA", inplace = True)
each_participant_rating_df.reset_index(drop = True, inplace = True)
each_participant_rating_df
# each_participant_rating_df.to_csv("How often participants gave unfair labels w.r.t each participant.csv")

## How often participants suggested weight changes w.r.t each participant

In [ ]:
feedback_group = feedback_df[(feedback_df["Function"] == "select_reject_application")]
feedback_group = pd.DataFrame(feedback_group)
for idx in feedback_group.index:
    feedback_group.at[idx,"Value"] = str(feedback_group.loc[idx]["Value"])

In [ ]:
labels_with_weights_all = feedback_group.loc[[idx  for idx in feedback_group.index if feedback_group.loc[idx]['Value']!=[]]]
# labels_with_weights_all = feedback_group[[True if isinstance(val,str) and val!="[]" else False for val in feedback_group["Value"]]]
labels_with_weights_unfair =labels_with_weights_all[labels_with_weights_all['Attribute']=='unfair']
labels_with_weights_ids= labels_with_weights_unfair['ID']

labels_without_weights_all = feedback_group.loc[[idx  for idx in feedback_group.index if feedback_group.loc[idx]['Value']==[]]]
# labels_without_weights_all = feedback_group[[True if isinstance(val,str) and val=="[]" else False for val in feedback_group["Value"]]]
labels_without_weights_unfair = labels_without_weights_all[labels_without_weights_all['Attribute']=='unfair']
labels_without_weights_ids = labels_without_weights_all['ID']


In [ ]:
participants_weight_group= feedback_group.groupby(by = ["ID","Value"])

participant_count = participants_weight_group.size().unstack(fill_value=0).stack()

data = []
for i in range(0,len(participant_count.index)):    
    # if participant_count.index[i][1] in feedback_df[(feedback_df["Function"] == "OKBUTTON_CLICKED_DECIDE_MODAL_Applications_List") & (feedback_df["Value"] != "[]")]["Value"].unique():
    if participant_count.index[i][1] in labels_with_weights_all["Value"].unique():    
        data.append(
        {
            "Participant" : participant_count.index[i][0],
            "Weight" : participant_count.index[i][1],
            "Count": participant_count.values[i]
        }
        )
    
participant_weight_df = pd.DataFrame(data)
participant_weight_df

In [ ]:
data = []

for participant in participant_weight_df["Participant"].unique():
    
    data.append(
    {
        "Participant" : participant,
        "Total_Weight_Changes" : participant_weight_df[participant_weight_df["Participant"]== participant ].Count.sum(),
    }
    )
    
    
each_participant_weight_df = pd.DataFrame(data)

each_participant_weight_df.sort_values(by='Total_Weight_Changes', ascending=False, inplace = True)
each_participant_weight_df.fillna("NA", inplace = True)
each_participant_weight_df.reset_index(drop = True, inplace = True)
each_participant_weight_df
# each_participant_weight_df.to_csv("How often participants suggested weight changes w.r.t each participant.csv")

# Can't delete these entries because the participant is labeling different applications in these entries.

## How many participants provided feedback for the same application more than once

In [ ]:
feedback_group = feedback_df[feedback_df["Function"] == "select_reject_application"]
participants_checked_app_multiple_times=[]
for part in feedback_group['ID'].unique():
    part_feedback = feedback_group[feedback_group["ID"]== part]
    if len(part_feedback)> len(part_feedback['App ID'].unique()):
        participants_checked_app_multiple_times.append(part)
len(participants_checked_app_multiple_times)

## How many participants suggested a weight change for each attribute and average weight percentage change of weight

In [ ]:
for idx in feedback_df.index:
    value = feedback_df.loc[idx]['Value']
    if value == []:
        feedback_df.at[idx,'Value'] = {}
    else:
        transf_value = {}
        for el in value:
            transf_value[el['attribute']] = el['value']
        feedback_df.at[idx,'Value'] = transf_value
feedback_df

In [ ]:
fdbck_with_weights = feedback_df[[True if len(val) else False for val in feedback_df["Value"]]]

feature_weights_dict = {}
for i in range(49):
    feature_weights_dict[applications_df['ft_name_'+str(i)].tolist()[0]] = applications_df['ft_weight_'+str(i)].tolist()[0]
    
total_clickcs_per_attr ={}
number_part_per_attr = {}
part_appid_weight_ch_dict = {}
for idx in fdbck_with_weights.index:
    ch_weights = fdbck_with_weights.loc[idx]['Value']
    for attr in ch_weights:
        if attr not in total_clickcs_per_attr:
            total_clickcs_per_attr[attr] = 0
        if attr not in number_part_per_attr:
            number_part_per_attr[attr] = []
        if fdbck_with_weights.loc[idx]['ID'] not in part_appid_weight_ch_dict:
            part_appid_weight_ch_dict[fdbck_with_weights.loc[idx]['ID']] = {}
        if fdbck_with_weights.loc[idx]['App ID'] not in part_appid_weight_ch_dict[fdbck_with_weights.loc[idx]['ID']]:
             part_appid_weight_ch_dict[fdbck_with_weights.loc[idx]['ID']][fdbck_with_weights.loc[idx]['App ID']] = {}
        if attr not in part_appid_weight_ch_dict[fdbck_with_weights.loc[idx]['ID']][fdbck_with_weights.loc[idx]['App ID']]:
            part_appid_weight_ch_dict[fdbck_with_weights.loc[idx]['ID']][fdbck_with_weights.loc[idx]['App ID']][attr] = None
        part_appid_weight_ch_dict[fdbck_with_weights.loc[idx]['ID']][fdbck_with_weights.loc[idx]['App ID']][attr] = float(ch_weights[attr]) - feature_weights_dict[attr]
        if fdbck_with_weights.loc[idx]['ID'] not in number_part_per_attr[attr]:
            number_part_per_attr[attr].append(fdbck_with_weights.loc[idx]['ID'])
        total_clickcs_per_attr[attr] = total_clickcs_per_attr[attr] + 1
for attr in number_part_per_attr:
    number_part_per_attr[attr] = len(number_part_per_attr[attr])

av_weight_ch_dict = {}
for part in part_appid_weight_ch_dict:
    for appid in part_appid_weight_ch_dict[part]:
        for attr in part_appid_weight_ch_dict[part][appid]:
            if attr not in av_weight_ch_dict:
                av_weight_ch_dict[attr] = []
            av_weight_ch_dict[attr].append(part_appid_weight_ch_dict[part][appid][attr])
for attr in av_weight_ch_dict:
    av_weight_ch_dict[attr] = sum(av_weight_ch_dict[attr])/len(av_weight_ch_dict[attr]) 
    av_weight_ch_dict[attr] = (av_weight_ch_dict[attr]/feature_weights_dict[attr])*100 if feature_weights_dict[attr] else np.inf
    
data = []
for attr in total_clickcs_per_attr:
    data.append({'attribute':attr, 
                 # 'Total_Weight_Changes':total_clickcs_per_attr[attr],                 
                 'Number of participants clicked':number_part_per_attr[attr],
                'Av. Perc. Weight Ch. %':av_weight_ch_dict[attr]})

weight_changes_df = pd.DataFrame(data)
weight_changes_df.sort_values(by='Number of participants clicked', ascending=False, inplace = True)
weight_changes_df.fillna("NA", inplace = True)
weight_changes_df.reset_index(drop = True, inplace = True)
weight_changes_df
weight_changes_df.to_csv("How many participants suggested a weight change for each attribute and average weight percentage change of weight.csv")

## How  many times an application was rated by Users w.r.t each participant

In [ ]:
parti_app_rating_group= feedback_df.groupby(by = ["ID","App ID","Attribute"])

parti_app_rating_count = parti_app_rating_group.size().unstack(fill_value=0).stack()

data = []

app_id_list = feedback_df["App ID"].unique().tolist()
# app_id_list.remove(0)

for i in range(0,len(parti_app_rating_count.index)):
    if (parti_app_rating_count.index[i][1] in app_id_list) and (parti_app_rating_count.index[i][2] in feedback_df[feedback_df["Function"] == "select_reject_application"]["Attribute"].unique().tolist()):

        data.append(
        {
            "Participant" : parti_app_rating_count.index[i][0],
            "Application_Label" : (parti_app_rating_count.index[i][1],parti_app_rating_count.index[i][2]) ,
            "Application" : parti_app_rating_count.index[i][1],
            "Label" : parti_app_rating_count.index[i][2],
            "Count": parti_app_rating_count.values[i]
        }
        )
    
parti_app_rating_df = pd.DataFrame(data)
parti_app_rating_df.head(5)

In [ ]:
data = []

for app_label in parti_app_rating_df["Application_Label"].unique():
    
    data.append(
    {
        "App_Label" : app_label,
        "Total_Ratings" : parti_app_rating_df[parti_app_rating_df["Application_Label"]== app_label].Count.sum(),
        "Mean": round(parti_app_rating_df[parti_app_rating_df["Application_Label"]== app_label].Count.mean(),2),
        "Std": round(parti_app_rating_df[parti_app_rating_df["Application_Label"]== app_label].Count.std(),2),
        "Number of Participants Labeled": parti_app_rating_df[(parti_app_rating_df["Application_Label"]== app_label) & (parti_app_rating_df["Count"] != 0)].shape[0]}
    )
    
    
    
each_participant_app_rating_df = pd.DataFrame(data)

each_participant_app_rating_df.sort_values(by='Total_Ratings', ascending=False, inplace = True)
each_participant_app_rating_df.fillna(0, inplace = True)
each_participant_app_rating_df.reset_index(drop = True, inplace = True)

applications_list = []
label_list = []
for i in range(0,len(each_participant_app_rating_df)):
    applications_list.append(each_participant_app_rating_df["App_Label"][i][0])
    label_list.append(each_participant_app_rating_df["App_Label"][i][1])

each_participant_app_rating_df["Application"] = applications_list
each_participant_app_rating_df['Label'] = label_list
each_participant_app_rating_df = each_participant_app_rating_df.loc[:, ["Application","Label","Total_Ratings","Mean","Std","Number of Participants Labeled"]]

each_participant_app_rating_df.sort_values(by='Application', ascending=True, inplace = True)
each_participant_app_rating_df.reset_index(drop = True, inplace = True)

for index in each_participant_app_rating_df.index: 
    app_id = each_participant_app_rating_df.loc[index,"Application"]
    each_participant_app_rating_df.loc[index,"Predicted_decision"] = df[app_id]["Predicted_decision"]
    each_participant_app_rating_df.loc[index,"Prediction_Confidence_Accepted"] = df[app_id]["Prediction_Confidence_Accepted"]

each_participant_app_rating_df
#each_participant_app_rating_df.to_csv("How  many times an application was rated by Users w.r.t each participant.csv")